# NER 命名实体识别任务baseline
本示例仅提供完整的运行流程，供大家进行初步学习<br>
自行尝试须注意基本配置与文件目录的定义<br>
具体脚本内容可在相应.py文件内查看

### 1.基本配置
安装脚本依赖库<br>
每次重启notebook须重新安装

In [1]:
!pip install docopt

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=c6304696515f40640f18d5d0c4072c47c98c697cc208421e68e4d6b395a67c49
  Stored in directory: /home/ma-user/.cache/pip/wheels/6a/96/4d/255bd44721c1c199348fc6c9090b061b587eeaf5a7d6315c08
Successfully built docopt
You should consider upgrading via the '/home/ma-user/anaconda3/envs/PyTorch-1.10.2/bin/python3.7 -m pip install --upgrade pip' command.


切换至Baseline里的文件目录

In [2]:
%cd Baseline/model

/home/ma-user/work/Baseline/model


### 2.构建词典<br>
须在代码中替换训练集文件目录（示例：data/train.txt）<br>
生成的.json文件自动保存在vocab目录(示例：‘vocab/sent_vocab.json vocab/tag_vocab.json’)

In [27]:
!python utils.py
!python vocab.py  data/train.txt  vocab/sent_vocab.json  vocab/tag_vocab.json

The number of sentences in train.txt: 7


### 3.模型训练<br>
修改epoch数量可以通过修改代码中('--max-epoch=10')中的数字进行修改，如未使用gpu，删除最后的('--cuda')即可，其他具体参数可在run.py查看<br>
注意训练集文件目录

In [28]:
!python run.py train data/train.txt vocab/sent_vocab.json vocab/tag_vocab.json --max-epoch=10 --cuda

num of training examples: 5
num of development examples: 2
start training...
log: epoch 10, iter 10, 2492.6 words/sec, avg_loss 148.127067, time 1.3 sec
Reached 10 epochs, Save result model to ./model.pth


### 4.模型测试<br>
使用上一步训练的模型在测试集判分<br>
如需修改测试集，可替换test.txt，或修改.sh脚本中路径<br>
生成结果文件result.txt，可打开查看格式示例

In [29]:
!sh run.sh test

num of test samples: 1
start testing...
using device cuda
processed 49 tokens with 3 phrases; found: 0 phrases; correct: 0.
accuracy:  57.14%; precision:   0.00%; recall:   0.00%; FB1:   0.00
             DATE: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
              LOC: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
              ORG: precision:   0.00%; recall:   0.00%; FB1:   0.00  0


### 5.模型上传至ModelArts-AI应用
将训练好的模型导入ModelArts 将模型导入ModelArts，为后续推理测试、模型提交做准备。最后显示“所有模型导入完成”证明运行成功。

In [4]:
from modelarts.session import Session
from modelarts.model import Model
from modelarts.config.model_config import TransformerConfig,Params
!pip install json5
import json5
import re
import traceback
import random
import moxing as mox

try:
    session = Session()
    config_path = 'Baseline/model/config.json'  #注意config.json路径
    if mox.file.exists(config_path): # 判断一下是否存在配置文件，如果没有则不能导入模型
        model_location =  'Baseline/model'
        model_name = "modelname" #模型名称可自行定义
        load_dict = json5.loads(mox.file.read(config_path))
        model_type = load_dict['model_type']
        re_name = '_'+str(random.randint(0,1000))
        model_name += re_name
        runtime=load_dict['runtime']
        print("正在导入模型,模型名称：", model_name)
        model_instance = Model(
                     session, 
                     model_name=model_name,               # 模型名称
                     model_version="1.0.0",               # 模型版本
                      source_location_type='LOCAL_SOURCE',
                     source_location=model_location,      # 模型文件路径
                     model_type=model_type,# 模型类型
                     runtime=runtime
                     )

    print("所有模型导入完成")
except Exception as e:
    print("发生了一些问题，请看下面的报错信息：") 
    traceback.print_exc()
    print("模型导入失败")

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
You should consider upgrading via the '/home/ma-user/anaconda3/envs/PyTorch-1.10.2/bin/python3.7 -m pip install --upgrade pip' command.
正在导入模型,模型名称： model_888
Successfully create bucket modelarts-cn-north-4-46c3fe9b 


INFO:obs:Successfully upload file /home/ma-user/work/Baseline/model to OBS modelarts-cn-north-4-46c3fe9b/model-0717-044831
INFO:obs:Successfully upload file /home/ma-user/work/Baseline/model/.ipynb_checkpoints to OBS modelarts-cn-north-4-46c3fe9b/model-0717-044831/model
INFO:obs:Successfully upload file /home/ma-user/work/Baseline/model/.ipynb_checkpoints to OBS modelarts-cn-north-4-46c3fe9b/model-0717-044831/model
INFO:obs:Successfully upload file /home/ma-user/work/Baseline/model/.ipynb_checkpoints to OBS modelarts-cn-north-4-46c3fe9b/model-0717-044831/model
INFO:obs:Successfully upload file /home/ma-user/work/Baseline/model/.ipynb_checkpoints to OBS modelarts-cn-north-4-46c3fe9b/model-0717-044831/model
INFO:obs:Successfully upload file /home/ma-user/work/Baseline/model/.ipynb_checkpoints to OBS modelarts-cn-north-4-46c3fe9b/model-0717-044831/model
INFO:obs:Successfully upload file /home/ma-user/work/Baseline/model to OBS modelarts-cn-north-4-46c3fe9b/model-0717-044831
INFO:obs:Succe

Successfully upload file /home/ma-user/work/Baseline/model to OBS modelarts-cn-north-4-46c3fe9b/model-0717-044831
Successfully upload model files from /home/ma-user/work/Baseline/model to obs path /modelarts-cn-north-4-46c3fe9b/model-0717-044831.
The model source location is https://modelarts-cn-north-4-46c3fe9b.obs.cn-north-4.myhuaweicloud.com/model-0717-044831/model
publishing
published
所有模型导入完成
